In [ ]:
import os
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


In [ ]:
def event_counter(data_frames, at_time):
    # team 100(blue)
    team_100_events = {
        'CHAMPION_KILL': 0, # int
        'CHAMPION_KILL_ASSIST' : 0, # int
        'BUILDING_KILL': 0, # int
        'HORDE_KILL' : 0, # int
        'RIFTHERALD_KILL' : 0, # bool
        'DRAGON_KILL' : 0, # int
        'ElDER_DRAGON_KILL' : 0, # bool
        'BARON_NASHOR_KILL' : 0, # bool
        'WARD_PLACED': 0, # int
        'WARD_KILL': 0, # int
        'TURRET_PLATE_DESTROYED' : 0, # int
    }
    # team 200(red)
    team_200_events = {
        'CHAMPION_KILL': 0, # int 
        'CHAMPION_KILL_ASSIST' : 0, # int 
        'BUILDING_KILL': 0, # int
        'HORDE_KILL' : 0, # int
        'RIFTHERALD_KILL' : 0, # int
        'DRAGON_KILL' : 0, # int
        'ElDER_DRAGON_KILL' : 0, # bool
        'BARON_NASHOR_KILL' : 0, # bool
        'WARD_PLACED': 0, # int
        'WARD_KILL': 0, # int
        'TURRET_PLATE_DESTROYED' : 0, # int
    }
    winning_team = None
    
    for event in data_frames[-1]['events']:
        # game end
        if event['type'] == 'GAME_END':
            winning_team = event['winningTeam']

    # 특정 시간까지 주요 이벤트 카운트
    for data_frame in data_frames[:at_time]:
        for event in data_frame['events']:
            # champion kill
            if event['type'] == 'CHAMPION_KILL':
                # killer
                if event['killerId'] in [1,2,3,4,5]:
                    team_100_events['CHAMPION_KILL'] += 1
                    if "assistingParticipantIds" in event.keys():
                        team_100_events['CHAMPION_KILL_ASSIST'] += len(event["assistingParticipantIds"])
                elif event['killerId'] in [6,7,8,9,10]:
                    team_200_events['CHAMPION_KILL'] += 1
                    if "assistingParticipantIds" in event.keys():
                        team_200_events['CHAMPION_KILL_ASSIST'] += len(event["assistingParticipantIds"])
            elif event['type'] == 'BUILDING_KILL':
                if event['teamId'] == 100:
                    team_200_events['BUILDING_KILL'] += 1
                elif event['teamId'] == 200:
                    team_100_events['BUILDING_KILL'] += 1
            elif event['type'] == 'WARD_PLACED':
                if event['creatorId'] in [1,2,3,4,5]:
                    team_100_events['WARD_PLACED'] += 1
                elif event['creatorId'] in [6,7,8,9,10]:
                    team_200_events['WARD_PLACED'] += 1
            elif event['type'] == 'WARD_KILL':
                if event['killerId'] in [1,2,3,4,5]:
                    team_100_events['WARD_KILL'] += 1
                elif event['killerId'] in [6,7,8,9,10]:
                    team_200_events['WARD_KILL'] += 1
            elif event['type'] == 'TURRET_PLATE_DESTROYED':
                if event['teamId'] == 100:
                    team_200_events['TURRET_PLATE_DESTROYED'] += 1
                elif event['teamId'] == 200:
                    team_100_events['TURRET_PLATE_DESTROYED'] += 1
            elif event['type'] == 'ELITE_MONSTER_KILL':
                if event['killerId'] in [1,2,3,4,5]:
                    # 용
                    if event['monsterType'] == 'DRAGON':
                        team_100_events['DRAGON_KILL'] += 1
                    # 공허유충
                    elif event['monsterType'] == 'HORDE':
                        team_100_events['HORDE_KILL'] += 1
                    # 전령
                    elif event['monsterType'] == 'RIFTHERALD':
                        team_100_events['RIFTHERALD_KILL'] = 1
                    # 내셔남작
                    elif event['monsterType'] == 'BARON_NASHOR':
                        team_100_events['BARON_NASHOR_KILL'] += 1
                    # 장로
                    elif event['monsterType'] == 'ELDER_DRAGON':
                        team_100_events['ELDER_DRAGON_KILL'] += 1
                elif event['killerId'] in [6,7,8,9,10]:
                    # 용
                    if event['monsterType'] == 'DRAGON':
                        team_200_events['DRAGON_KILL'] += 1
                    # 공허유충
                    elif event['monsterType'] == 'HORDE':
                        team_200_events['HORDE_KILL'] += 1
                    # 전령
                    elif event['monsterType'] == 'RIFTHERALD':
                        team_200_events['RIFTHERALD_KILL'] = 1
                    # 내셔남작
                    elif event['monsterType'] == 'BARON_NASHOR':
                        team_200_events['BARON_NASHOR_KILL'] += 1
                    # 장로
                    elif event['monsterType'] == 'ELDER_DRAGON':
                        team_200_events['ELDER_DRAGON_KILL'] += 1
    return (team_100_events, team_200_events, winning_team)

In [ ]:
# team numierics integration
def numeric_integration(data_frames, at_time):
    team_100_numeric = {
        'totalGold': 0,
        'ToChampDamage': 0,
        'DamageTaken': 0,
        'sum_of_level': 0,
    }
    team_200_numeric = {
        'totalGold': 0,
        'ToChampDamage': 0,
        'DamageTaken': 0,
        'sum_of_level': 0,
    }
    # data_frames[at_time]
    for participant in data_frames[at_time]['participantFrames'].keys():
        if int(participant) in [1,2,3,4,5]:
            team_100_numeric['totalGold'] += data_frames[at_time]['participantFrames'][participant]['totalGold']
            team_100_numeric['ToChampDamage'] += data_frames[at_time]['participantFrames'][participant]['damageStats']['totalDamageDoneToChampions']
            team_100_numeric['DamageTaken'] += data_frames[at_time]['participantFrames'][participant]['damageStats']['totalDamageTaken']
            team_100_numeric['sum_of_level'] += data_frames[at_time]['participantFrames'][participant]['level']
        elif int(participant) in [6,7,8,9,10]:
            team_200_numeric['totalGold'] += data_frames[at_time]['participantFrames'][participant]['totalGold']
            team_200_numeric['ToChampDamage'] += data_frames[at_time]['participantFrames'][participant]['damageStats']['totalDamageDoneToChampions']
            team_200_numeric['DamageTaken'] += data_frames[at_time]['participantFrames'][participant]['damageStats']['totalDamageTaken']
            team_200_numeric['sum_of_level'] += data_frames[at_time]['participantFrames'][participant]['level']

    return (team_100_numeric, team_200_numeric)
#    data_frame[10]['participantFrames']['1']['totalGold']
#data_frame[10]['participantFrames']['1'].keys()
#data_core['frames'][10]['participantFrames']['1']['damageStats'].keys()

# 'totalDamageDone', 'totalDamageDoneToChampions', 'totalDamageTaken', 'sum_of_level'

In [ ]:
# data dir
json_dir = './data/match_timeline/'
json_files = os.listdir(json_dir)

len(json_files)

In [ ]:
# constants
data_length_list = [8,10,15,20,25]

# inverse for loop
for data_length in data_length_list[::-1]:
    print(data_length)
    data_list = []
    for json_file in tqdm(json_files):
        with open(json_dir + json_file) as f:
            data = json.load(f)
        match_id = data['metadata']['matchId']
        gameLength = len(data['info']['frames'])-2

        # delete abnormal data
        if gameLength >= data_length:
            data_frames = data['info']['frames']
            t100_events, t200_events, winning_team = event_counter(data_frames, data_length)
            t100_numerics, t200_numerics = numeric_integration(data_frames, data_length)
            # stack data
            data_list.append([[match_id] + list(t100_events.values()) + \
                            list(t100_numerics.values()) + list(t200_events.values()) + \
                            list(t200_numerics.values()) + [gameLength, winning_team]])
        
    cols =  [ 'match_id' ] + \
            [ 't100_events_' + key for key in t100_events.keys()] + \
            [ 't100_numerics_' + key for key in t100_numerics.keys()] + \
            [ 't200_events_' + key for key in t200_events.keys()] + \
            [ 't200_numerics_' + key for key in t200_numerics.keys()] + \
            [ 'GameLength', 'winning_team' ]
    
    dataArray = np.array(data_list).reshape(-1,33)
     # save as csv
    df = pd.DataFrame(dataArray, columns=cols)
    df.to_csv(f'./data/data_{data_length}min.csv', index=False)
